In [15]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import pandas as pd
import re

In [16]:
fin_dict = dict()

In [17]:
df = pd.read_excel('./target.xlsx')
address_list = df['주소'].values

In [18]:
#건축물대장 열람신청 입구
url = 'https://www.gov.kr/main?a=AA020InfoCappViewApp&CappBizCD=15000000098'

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
options = Options()
options.add_argument('--start-maximized') #big size
options.add_argument(f'user-agent={headers}')
# options.add_argument('headless') #창안띄우고 진행
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


### 반자동화

In [19]:
#로그인 
driver.get(url)
time.sleep(0.5)
#신청하기버튼
ele = driver.find_element_by_css_selector('#applyBtn > a')
ele.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#applyBtn > a"}
  (Session info: chrome=89.0.4389.114)


### 함수화

In [6]:
def find_owner_crawling(address, dong_length = False, ho_length = False, dong = 1, ho = 1):
    '''
    아파트 주소를 제공받아 소유주를 확인하는 코드
    총 3단계로 이뤄져야함
    1단계 해당 주소의 동개수 파악
    2단계 해당 주소의 동의 호수 파악
    3단계 각 동-호수별로 소유주 파악 
    '''
    sleep_time = 0.5
    #로그인 
    driver.get(url)
    time.sleep(sleep_time)
    #신청하기버튼
    ele = driver.find_element_by_css_selector('#applyBtn > a')
    ele.click()
    wait = WebDriverWait(driver, 360).until(EC.element_to_be_clickable((By.CLASS_NAME, 'data-row-span-2')), 'Timed out waiting for loading')
    
    #열람버튼클릭
    ele = driver.find_element_by_css_selector('body > div.contentsWrap > div > div.form-inner > div.element-tab > a:nth-child(2)')
    ele.click()
    wait = WebDriverWait(driver, 360).until(EC.element_to_be_clickable((By.CLASS_NAME, 'data-row-span-2')), 'Timed out waiting for loading')

    #아파트 연립주택선택
    ele = driver.find_element_by_css_selector('#건축물관리대장발급신청서_IN-건축물관리대장발급신청서_입력항목_공통항목_대장구분_\.라디오코드2')
    ele.click()
    #전유부 선택
    ele = driver.find_element_by_css_selector('#건축물관리대장발급신청서_IN-건축물관리대장발급신청서_입력항목_공통항목_대장종류_\.라디오코드13')
    ele.click()

    #검색클릭
    ele = driver.find_element_by_css_selector('#주소검색')
    ele.click()
    time.sleep(sleep_time)

    #팝업창이동
    all_windows = driver.window_handles 
    parent_window = all_windows[0] 
    child_window = all_windows[1] 
    driver.switch_to.window(child_window) 
    time.sleep(sleep_time)

    #검색입력
    ele = driver.find_element_by_css_selector('#txtRoad')
    ele.send_keys(address)
    time.sleep(sleep_time)

    #검색클릭
    ele =driver.find_element_by_css_selector('#frm_popup > fieldset > div > div > span > button')
    ele.click()
    time.sleep(sleep_time)

    #도로명주소클릭
    ele = driver.find_element_by_css_selector('#pop-wrap > div > div.cont-box.addr-search-new > div > div.address-result-list > a > dl > dd > div:nth-child(1)')
    ele.click()

    #행정처리기관클릭
    ele = driver.find_element_by_css_selector('#frm_popup > fieldset > div.search-btm-area > div.address-result-list.land > a:nth-child(2) > dl > dd > div')
    ele.click()

    #복귀
    driver.switch_to.window(parent_window) 
    
    #민원신청하기 클릭 - 동선택까지 기다리기
    ele = driver.find_element_by_css_selector('#btn_end')
    ele.click()
    time.sleep(sleep_time)
    wait = WebDriverWait(driver, 360).until(EC.element_to_be_clickable((By.CLASS_NAME, 'data-row-span-2')), 'Timed out waiting for loading')
    time.sleep(sleep_time)

    #팝업창이동(동선택)
    all_windows = driver.window_handles 
    parent_window = all_windows[0] 
    child_window = all_windows[1] 
    driver.switch_to.window(child_window) 
    time.sleep(sleep_time)

    if dong_length == True:
        #전체 동수파악
        ele = driver.find_element_by_css_selector('body > div > div > div > table > tbody')
        dong_pattern = re.compile('[0-9]+동')
        dong_length = len([item.split(' ')[1] for item in ele.text.split('\n')])
        #복귀
        driver.close()
        driver.switch_to.window(parent_window) 
        return dong_length

    #동 선택(지정안해주면 1)
    ele = driver.find_element_by_css_selector(f'body > div > div > div > table > tbody > tr:nth-child({dong+1}) > td:nth-child(3) > span > button')
    ele.click()

    #복귀
    driver.switch_to.window(parent_window) 

    #민원신청하기 클릭 - 호선택까지 기다리기
    ele = driver.find_element_by_css_selector('#btn_end')
    ele.click()
    time.sleep(sleep_time)
    wait = WebDriverWait(driver, 360).until(EC.element_to_be_clickable((By.CLASS_NAME, 'data-row-span-2')), 'Timed out waiting for loading')
    time.sleep(sleep_time)

    #팝업창이동(호선택)
    all_windows = driver.window_handles 
    parent_window = all_windows[0] 
    child_window = all_windows[1] 
    driver.switch_to.window(child_window) 
    time.sleep(sleep_time)

    if ho_length == True:
        #전체 호수파악
        ele = driver.find_element_by_css_selector('body > div > div > div > table > tbody')
        ho_length = len([item.split(' ')[1] for item in ele.text.split('\n')])
        

    #호 클릭(지정안해주면 1)
    ele = driver.find_element_by_css_selector(f'body > div > div > div > table > tbody > tr:nth-child({ho+1}) > td:nth-child(3) > span > button')
    ele.click()

    #복귀
    driver.switch_to.window(parent_window) 

    #민원신청하기
    ele = driver.find_element_by_css_selector('#btn_end')
    ele.click()
    time.sleep(1)
    wait = WebDriverWait(driver, 180).until(EC.element_to_be_clickable((By.ID, 'srch')), 'Timed out waiting for loading')
    time.sleep(1)
    
    if ho_length == True:
        return ho_length
       

In [7]:
#함수 실행. 아파트 1개씩 돌아가는 코드
address = address_list[0]
#아파트내 동개수 파악
dong_length = find_owner_crawling(address, dong_length = True, dong = 1, ho = 1)
#동별 호개수 파악후 서류신청
for dong in tqdm(range(dong_length)):
    ho_length = find_owner_crawling(address, ho_length = True, dong = dong, ho = 1) #각 동의 첫번째는 진행함
    for ho in tqdm(range(1,ho_length)):
        find_owner_crawling(address, dong = dong, ho = ho)

UnexpectedAlertPresentException: Alert Text: 민원처리기관인 [의정부시] 시스템의 문제로 현재 민원을 신청할 수 없습니다.
시스템 담당자가 해당 처리기관인 [의정부시]와 협조하여 신속히 조치하도록 하겠습니다.
지속적인 서비스 불가시 공지사항을 확인해 주십시요.
Message: unexpected alert open: {Alert text : 민원처리기관인 [의정부시] 시스템의 문제로 현재 민원을 신청할 수 없습니다.
시스템 담당자가 해당 처리기관인 [의정부시]와 협조하여 신속히 조치하도록 하겠습니다.
지속적인 서비스 불가시 공지사항을 확인해 주십시요.}
  (Session info: chrome=89.0.4389.114)


In [ ]:
#동별돌아가는 코드 (필요시 주석해제 후 개별실행)
# dong = 1
# ho_length = find_owner_crawling(address, ho_length = True, dong = dong, ho = 1) 
# for ho in tqdm(range(1,ho_length)):
#     find_owner_crawling(address, dong = dong, ho = ho)

### 저장

In [238]:
#표제부일때 
#저장위치
items = dict()
items['명칭'] = []
items['호명칭'] = []
items['면적'] = []
items['소유자'] = []
items['소유자_주소'] = []
items['소유자_주민번호'] = []
items['소유자2'] = [] 
items['소유자2_주소'] = []
items['소유자2_주민번호'] = []

#다음페이지
pages = [2,4,5,6,7,8,9]
for page in pages:
    ele = driver.find_element_by_css_selector(f"#EncryptionAreaID_0 > div.pagination > ul > li:nth-child({page}) > a")
    ele.click()
    time.sleep(0.5)
    wait = WebDriverWait(driver, 360).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ttl')), 'Timed out waiting for loading')
    #엑셀화 작업본
    #첫번째창
    for item_num in range(1,11):
        ele = driver.find_element_by_css_selector(f'#EncryptionAreaID_0 > div.tbl-list.border > table > tbody > tr:nth-child({item_num}) > td.cs-state > p:nth-child(2) > span > a')
        ele.click()
        time.sleep(1)

        #팝업창이동
        all_windows = driver.window_handles 
        parent_window = all_windows[0] 
        child_window = all_windows[1] 
        driver.switch_to.window(child_window) 
        time.sleep(1)

        #데이터가져오기
        ele = driver.find_element_by_css_selector('#EncryptionAreaID_0 > table:nth-child(6) > tbody > tr:nth-child(3) > td.L_R1B1.P_R1')
        명칭 = ele.text
        ele = driver.find_element_by_css_selector('#EncryptionAreaID_0 > table:nth-child(6) > tbody > tr:nth-child(3) > td.L_B1.P_R1')
        호명칭 = ele.text
        ele = driver.find_element_by_css_selector('#main > tbody > tr:nth-child(5) > td.L_R1B1.P_RM')
        면적 = ele.text

        ele = driver.find_element_by_css_selector('#main > tbody > tr:nth-child(5) > td:nth-child(6)')
        소유자 = ele.text
        ele = driver.find_element_by_css_selector('#main > tbody > tr:nth-child(5) > td:nth-child(7)')
        소유자_주소 = ele.text
        ele = driver.find_element_by_css_selector('#main > tbody > tr:nth-child(6) > td:nth-child(6)')
        소유자_주민번호 = ele.text

        items['명칭'].append(명칭) 
        items['호명칭'].append(호명칭) 
        items['면적'].append(면적) 
        items['소유자'].append(소유자) 
        items['소유자_주소'].append(소유자_주소) 
        items['소유자_주민번호'].append(소유자_주민번호)

        try:
            ele = driver.find_element_by_css_selector('#main > tbody > tr:nth-child(7) > td:nth-child(2)')
            소유자2 = ele.text
            ele = driver.find_element_by_css_selector('#main > tbody > tr:nth-child(7) > td.L_R1B1.P_LM.T_WRAP')
            소유자2_주소 = ele.text
            ele = driver.find_element_by_css_selector('#main > tbody > tr:nth-child(8) > td:nth-child(6)')
            소유자2_주민번호 = ele.text

            items['소유자2'].append(소유자2) 
            items['소유자2_주소'].append(소유자2_주소) 
            items['소유자2_주민번호'].append(소유자2_주민번호)
        except :
            items['소유자2'].append(np.nan) 
            items['소유자2_주소'].append(np.nan) 
            items['소유자2_주민번호'].append(np.nan)

        #복귀
        driver.close()
        driver.switch_to.window(parent_window)
#저장
items_df = pd.DataFrame(items)
items_df.to_excel('test.xlsx')